<a href="https://colab.research.google.com/github/tleitch/rethnicity/blob/main/rethnicity_singlechar_distill_fullname_aligned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Note

In [ ]:
# model on fullnames but make sure that last names aligned at the same position
# this is to overcome the fullname model only focus on firstname part

# Preprocess the data

In [ ]:
# change keras default GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="0" # first gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="1" # second gpu
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # runs in cpu

In [ ]:
# pip install keras-tuner

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 19 18:50:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/

/content/drive/MyDrive


In [5]:
import tensorflow.keras as keras
import tensorflow as tf
print(keras.__version__)
#print(tf.__version__)
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import string

2.8.0


In [6]:
df = pd.read_csv("nmzpAgeSexFL.csv")

In [7]:
df=df[df['race'].isin([1,2,3,4,5])]

#dfW=df[df['race'].isin([5])]
#dfB=df[df['race'].isin([3])]
#dfH=df[df['race'].isin([4])]
#dfA=df[df['race'].isin([2])]
#dfAI=df[df['race'].isin([1])]

In [8]:
# create ASCII dictionary
chars = ['E'] + [chr(i) for i in range(97,123)] + [' ', 'U']
id2char = {i:j for i,j in enumerate(chars)}
char2id = {j:i for i,j in enumerate(chars)}

In [9]:
# the characters here are all ASCII, good
# for name in df['name_combine'].tolist():
#     namechars = list(name)
#     for nc in namechars:
#         if nc not in char2id:
#             print(nc)

In [9]:
def name2id(name, l = 10):
    ids = [0] * l
    for i, c in enumerate(name):
        if i < l:
            if c.isalpha():
                ids[i] = char2id.get(c, char2id['U'])
            elif c in string.punctuation:
                ids[i] = char2id.get(c, char2id[' '])
            else:
                ids[i] = char2id.get(c, char2id['U'])
    return ids

In [ ]:
# convert the output (y) from 2-5 to 0-3
#y = [i-2 for i in y]

# Train the models

In [10]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional

num_words = len(id2char)
feature_len = 20 # cut texts after this number of words (among top max_features most common words)
batch_size = 512


num_classes = 2 # np.max(y_train) + 1
print(num_classes, 'classes')


2 classes


In [11]:
# set white to 0 and other to 1
df['race2'] = np.where(df['race']==5, 0, 1)
df.head()

,Unnamed: 0,name_first,name_last,zip5,sex,race,age,race2
0,1,Elizabeth,Walker,32643,F,5,59.049315,0
1,2,Alton,Palmer,32643,M,5,77.854795,0
2,3,Alicia,Mc Cleod,32607,F,3,54.402740,1
3,4,Dale,Scarborough,32643,M,5,69.868493,0
4,5,Daniel,Walker,32640,M,5,65.572603,0


In [12]:
X = [name2id(fn.lower()) + name2id(ln.lower()) for fn, ln in zip(df['name_first'], df['name_last'])]
y = [int(i) for i in df['race2'].tolist()]

In [ ]:
# Use full data set to train saved model


X_train = sequence.pad_sequences(X, maxlen=feature_len)
y_train = keras.utils.to_categorical(y, num_classes)
print('y_train shape:', y_train.shape)
print('X_train shape:', X_train.shape)

model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='sigmoid'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.2, verbose=1, callbacks=[callback])
#score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)


model.save('fullname_aligned_distillwhite.h5', include_optimizer=False)

y_train shape: (12988473, 2)
X_train shape: (12988473, 20)
Epoch 1/10
  561/20295 [..............................] - ETA: 1:13:57 - loss: 0.5563 - accuracy: 0.7299

In [ ]:
# remove white and create new race code to use for running against black
df=df[df['race']!=5]
df['race2'] = np.where(df['race']==3, 0, 1)
df.head()
X = [name2id(fn.lower()) + name2id(ln.lower()) for fn, ln in zip(df['name_first'], df['name_last'])]
y = [int(i) for i in df['race2'].tolist()]

In [ ]:
# Use full data set to train saved model


X_train = sequence.pad_sequences(X, maxlen=feature_len)
y_train = keras.utils.to_categorical(y, num_classes)
print('y_train shape:', y_train.shape)
print('X_train shape:', X_train.shape)

model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='sigmoid'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.2, verbose=1, callbacks=[callback])
#score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)


model.save('fullname_aligned_distillblack.h5', include_optimizer=False)

In [ ]:
# remove black and create new race code to use for running against hispanic
df=df[df['race']!=3]
df['race2'] = np.where(df['race']==4, 0, 1)
df.head()
X = [name2id(fn.lower()) + name2id(ln.lower()) for fn, ln in zip(df['name_first'], df['name_last'])]
y = [int(i) for i in df['race2'].tolist()]

In [ ]:
# Use full data set to train saved model


X_train = sequence.pad_sequences(X, maxlen=feature_len)
y_train = keras.utils.to_categorical(y, num_classes)
print('y_train shape:', y_train.shape)
print('X_train shape:', X_train.shape)

model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='sigmoid'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.2, verbose=1, callbacks=[callback])
#score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)


model.save('fullname_aligned_distillhispanic.h5', include_optimizer=False)

In [ ]:
# remove hispanic and create new race code to use for running against asian
df=df[df['race']!=4]
df['race2'] = np.where(df['race']==2, 0, 1)
df.head()
X = [name2id(fn.lower()) + name2id(ln.lower()) for fn, ln in zip(df['name_first'], df['name_last'])]
y = [int(i) for i in df['race2'].tolist()]

In [ ]:
# Use full data set to train saved model


X_train = sequence.pad_sequences(X, maxlen=feature_len)
y_train = keras.utils.to_categorical(y, num_classes)
print('y_train shape:', y_train.shape)
print('X_train shape:', X_train.shape)

model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='sigmoid'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_split=0.2, verbose=1, callbacks=[callback])
#score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)


model.save('fullname_aligned_distillasian.h5', include_optimizer=False)

# Tune the Model

In [1]:
model.summary()

NameError: ignored

In [ ]:
model.get_config()

In [ ]:
model.optimizer

In [ ]:
from keras import backend as K
K.eval(model.optimizer.lr)

In [ ]:
# from google.colab import files
# files.download('fullname_2_ethnicity_bilstm.h5') 